In [1]:
%matplotlib inline
from sklearn.model_selection import train_test_split

def multi_task_train_test_split(Xs,Ys,train_size=0.5):
    Xs_train = []
    Ys_train = []
    Xs_test = []
    Ys_test = []
    for t in range(len(Xs)):
        X_train, X_test, y_train, y_test = train_test_split(Xs[t], np.squeeze(Ys[t]), train_size=train_size)
        Xs_train.append(X_train)
        Xs_test.append(X_test)
        Ys_train.append(y_train)
        Ys_test.append(y_test)
    return Xs_train, Xs_test, Ys_train, Ys_test

from ELLA import ELLA
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from scipy.linalg import norm
import numpy as np

T = 20
d = 10
n = 100
k = 5
noise_var = .1

model = ELLA(d,k,Ridge,mu=1,lam=10**-5)

S_true = np.random.randn(k,T)
L_true = np.random.randn(d,k)
w_true = L_true.dot(S_true)

# make sure to add a bias term (it is not done automatically)
Xs = [np.hstack((np.random.randn(n,d-1), np.ones((n,1)))) for i in range(T)]
# generate the synthetic labels
Ys = [Xs[i].dot(w_true[:,i]) + noise_var*np.random.randn(n,) for i in range(T)]
# break into train and test sets
Xs_train, Xs_test, Ys_train, Ys_test = multi_task_train_test_split(Xs,Ys,train_size=0.5)

for t in range(T):
    model.fit(Xs_train[t], Ys_train[t], t)
print("Average explained variance score", np.mean([model.score(Xs_test[t], Ys_test[t], t) for t in range(T)]))

# Try out a classification problem
Ys_binarized_train = [Ys_train[i] > 0 for i in range(T)]
Ys_binarized_test = [Ys_test[i] > 0 for i in range(T)]

model = ELLA(d,k,LogisticRegression,mu=1,lam=10**-5)
for t in range(T):
    model.fit(Xs_train[t], Ys_binarized_train[t], t)

print("Average classification accuracy", np.mean([model.score(Xs_test[t], Ys_binarized_test[t], t) for t in range(T)]))

Average explained variance score 0.8304393148382092
Average classification accuracy 0.8819999999999999


In [18]:
print(np.array(Xs).shape)
print(np.array(Ys).shape)

(20, 100, 10)
(20, 100)


In [2]:
from scipy.io import loadmat
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

data = loadmat('landminedata.mat')

Xs_lm = []
Ys_lm = []
for t in range(data['feature'].shape[1]):
    X_t = data['feature'][0,t]
    Xs_lm.append(np.hstack((X_t,np.ones((X_t.shape[0],1)))))
    Ys_lm.append(data['label'][0,t] == 1.0)

d = Xs_lm[0].shape[1]
k = 1

Xs_lm_train, Xs_lm_test, Ys_lm_train, Ys_lm_test = multi_task_train_test_split(Xs_lm,Ys_lm,train_size=0.5)
model = ELLA(d,k,LogisticRegression,{'C':10**0},mu=1,lam=10**-5)
for t in range(T):
    model.fit(Xs_lm_train[t], Ys_lm_train[t], t)

print(model.S)

print("Average AUC:", np.mean([roc_auc_score(Ys_lm_test[t],
                                             model.predict_logprobs(Xs_lm_test[t], t))
                               for t in range(1)]))

[[-0.32435098 -0.28570778 -0.2712983  -0.26299444 -0.29489066 -0.29085745
  -0.30831645 -0.2853992  -0.29059767 -0.29244561 -0.28854514 -0.30840084
  -0.29665186 -0.32287709 -0.30121575 -0.20919978 -0.18757392 -0.23616294
  -0.19422565 -0.18789317]]
Average AUC: 0.8597029383274136


In [14]:
len(Xs_lm)

29